<a href="https://colab.research.google.com/github/MsAsterisk1/ERM_GDRO/blob/main/civil_comments_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!nvidia-smi -L  

GPU 0: NVIDIA GeForce RTX 2060 (UUID: GPU-100f20af-1b36-5adc-0795-f06b020c050b)


In [3]:
# !git clone https://github.com/MsAsterisk1/ERM_GDRO.git
# %cd /content/ERM_GDRO

In [4]:
# !git pull

In [5]:
# !pip install transformers

In [3]:
import pandas as pd
from utils.process_data_utils import  get_CivilComments_DataLoaders, get_CivilComments_Datasets, get_CivilComments_df
from loss import ERMLoss, GDROLoss
from train_eval import train, evaluate, train_epochs
import torch
from models import BertClassifier

Now lets try getting Bert working

In [4]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cuda'

In [5]:
df = get_CivilComments_df()

In [11]:
df.head()

,id,comment_text,split,created_date,publication_id,parent_id,article_id,rating,funny,wow,...,disability_any,identity_any,num_identities,more_than_one_identity,na_gender,na_orientation,na_religion,na_race,na_disability,others
0,59856,haha you guys are a bunch of losers.,0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,...,0,0,0.0,False,1,1,1,1,1,1
1,239579,This is a great story. Man. I wonder if the pe...,0,2016-01-13 18:43:00.710664+00,6,239524.0,26650,approved,0,0,...,0,1,1.0,False,0,1,1,1,1,0
2,239592,"Angry trolls, misogynists and Racists"", oh my....",0,2016-01-13 19:48:45.619202+00,6,NaN,26795,approved,0,0,...,0,0,0.0,False,1,1,1,1,1,1
3,239593,Nice to some attempts to try to make comments ...,0,2016-01-13 19:56:52.277231+00,6,NaN,26795,approved,0,0,...,0,0,0.0,False,1,1,1,1,1,1
4,239607,Yet call out all Muslims for the acts of a few...,2,2016-01-13 20:50:07.430323+00,6,239581.0,26670,approved,0,0,...,0,1,2.0,True,1,1,0,1,1,0


In [43]:
datasets = get_CivilComments_Datasets(CC_df=df[:30], device=DEVICE)

In [44]:
tr,cv,test=  get_CivilComments_DataLoaders(datasets=datasets)

In [45]:
next(tr)[3]

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [18]:
a.shape

torch.Size([16])

This is how we would initialize model

In [12]:
model = BertClassifier().to(DEVICE)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.00005, weight_decay=0.01)
epochs = 5
loss_fn = ERMLoss(model, torch.nn.CrossEntropyLoss())
# loss_fn = GDROLoss(model, torch.nn.CrossEntropyLoss(), 0.01, 8, vector_subclass=True)

train_epochs(epochs, tr, test, model, loss_fn, optimizer, vector_subclass=True, verbose=True, num_subclasses=8)


Epoch 1 / 5
Epoch 2 / 5
Epoch 3 / 5
Epoch 4 / 5
Epoch 5 / 5


In [14]:
torch.save(model.state_dict(), './CC_ERM.wt')


In [9]:
model = BertClassifier().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00005, weight_decay=0.01)
epochs = 5
# loss_fn = ERMLoss(model, torch.nn.CrossEntropyLoss())
loss_fn = GDROLoss(model, torch.nn.CrossEntropyLoss(), 0.01, 9, vector_subclass=True)

train_epochs(epochs, tr, test, model, loss_fn, optimizer, vector_subclass=True, verbose=True, num_subclasses=8)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1 / 5


RuntimeError: CUDA out of memory. Tried to allocate 66.00 MiB (GPU 0; 6.00 GiB total capacity; 5.05 GiB already allocated; 0 bytes free; 5.29 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
torch.save(model.state_dict(), './CC_GDRO.wt')
